# CH8 TensorFlow 高级编程

## 8.3 影评的情感分类

我们想要解决的是IMDB影视评论情感分类问题。每条电影评论都是一个单词变量序列，需要将这些影评按照情感（积极或消极）进行分类。

该问题十分复杂，因为序列长度不尽相同；另外，输入符号中包含的词汇也多种多样。

要解决这一问题，模型需要学习输入序列中的长期依赖。

IMDB数据集包含25 000条高度极端化的电影评论（好或坏）作为训练集，另有同样数量的数据作为测试集。数据集由斯坦福研究人员收集，并在2011年的一篇论文里使用。当时，他们将数据集分为两等份，分别用于训练和测试。在这篇论文里，分类准确率达到了88.90%。

定义了我们的问题后，即可开始构建一个LSTM模型来对影评进行情感分类。可以为IMDB问题快速开发一个LSTM模型，并获得很好的分类准确率。

先导入模型需要的类和函数，并初始化随机数生成器为一个常数值，以确保可以方便复现实验结果：

In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

numpy.random.seed(7)

Using TensorFlow backend.


然后加载IMDB数据集。将数据集限制在5000词以内。另外，也将数据集分为训练集（50%）和测试集（50%）。

`imdb.load_data()`函数允许将数据集加载为神经网络和深度学习模型适用的格式。单词已被替换为整数，代表每个单词在该数据集中的有序频率。这样，每条影评中的句子就被转换为一个整数序列。

In [3]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)

D:\Python\Anaconda3\lib\site-packages\keras\datasets\imdb.py:49: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


17465344/17464789 [==============================] - 342s 20us/step


接下来，需要截短/填充每个输入序列，使它们长度相同，方便建模。

模型会学习输入中不包含任何信息的0值，因为虽然序列的内容并不一样长，但使用Keras进行计算时，输入数组必须拥有相同长度。

每条影评中的序列长度不同，所以我们将每条影评限制为500词，将长于500词的评论截短，短于500词的评论用0值填充。

In [4]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

现在，开始定义、编译并拟合我们的LSTM模型。

为解决情感分类问题，我们将使用词嵌入技术，即将单词映射到一个连续的向量空间中，语义相近的单词会被映射到该空间内的相邻点上。**词嵌入基于分布假设，即给定的上下文中出现的单词一定有相同的语义特征。**这样，每条影评将会被映射到一个实向量域内，单词间的语义相似性因此被转换为向量空间中的点之间的距离。Keras提供了一种方便的方式，可以将单词的整数表示用一个嵌入层转换为词嵌入。

In [5]:
# 下面定义嵌入向量的长度以及模型：
embedding_vector_length = 32
model = Sequential()

In [6]:
# 第一层为嵌入层，使用长度为32的向量表示每个单词：
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))

下一个层为LSTM层，含有100个记忆单元。最后，由于需要解决的是一个分类问题，我们使用仅含有一个神经元的Dense输出层和一个sigmoid激活函数进行二元预测。其中0和1分别代表问题中的两个类（好或坏）：

In [7]:
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

由于这是一个二元分类问题，所以使用binary_crossentropy函数作为loss函数，而optimizer函数采用adam优化算法：

In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# 我们只拟合3个训练时期，因为该问题会很快过拟合。将每64个影评分为一批，进行权重更新：
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64)

D:\Python\Anaconda3\lib\site-packages\keras\models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 594s 24ms/step - loss: 0.4257 - acc: 0.8019 - val_loss: 0.3355 - val_acc: 0.8608
Epoch 2/3
25000/25000 [==============================] - 586s 23ms/step - loss: 0.2916 - acc: 0.8845 - val_loss: 0.3156 - val_acc: 0.8721
Epoch 3/3
25000/25000 [==============================] - 541s 22ms/step - loss: 0.2682 - acc: 0.8939 - val_loss: 0.3107 - val_acc: 0.8723


In [10]:
# 然后，评估该模型在测试影评上的表现：
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.23%


## 8.4 添加一个卷积层
**可以在嵌入层后添加一维CNN和最大池化层，这些层可以将强化的特征馈给LSTM网络。**

In [ ]:
# 以下是我们的嵌入层：
model = Sequential()
model.add(Embedding(top_words,embedding_vector_length, input_length=max_review_length))

In [ ]:
# 可以添加一个卷积核大小（filter_length）为3的小卷积核卷积层，带有32个输出特征（nb_filter）：
model.add(Conv1D (padding="same", activation="relu", kernel_size=3, num_filter=32))

In [ ]:
# 接下来，添加一个池化层。最大池化的应用范围大小为2：
model.add(GlobalMaxPooling1D ())

In [ ]:
# 下一个层为LSTM层，含有100个记忆单元：
model.add(LSTM(100))

In [ ]:
# 最后的层为一个Dense输出层，含有一个神经元和一个sigmoid激活函数，
# 用来给出预测结果0或1，代表问题中的（好或坏）两个类（亦即这是一个二元分类问题）：
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# 完整源代码
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Conv1D, GlobalMaxPooling1D

# fix random seed for reproducibility
numpy.random.seed(7)

# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# create the model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D (padding="same", activation="relu", kernel_size=3, num_filter=32))
model.add(GlobalMaxPooling1D ())

model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))

## 8.7 TFLearn
TFLearn是一个函数库，它将许多TensorFlow的全新API封装成易用且熟悉的scikit-learn API。

**TensorFlow的本质是建立和执行一个图。**这是一个非常强大的概念，但对于初学者来说可能难以理解。

若将TFLearn作为外壳，我们只需要用到TensorFlow的三部分。
* **层**：这是一系列高级TensorFlow函数，允许你方便地构建复杂的图——从全连接层、卷积层和批归一化（BN）层，到损失函数和优化函数。
* **图操作**：这是一系列工具，用于训练、测试和在TensorFlow图上运行界面。
* **评估算子**：该算子封装了一个类中所有的scikit-learn接口，并提供了一种方便地构建和训练自定义TensorFlow模型的方式。评估算子的子类，如线性分类器、线性回归器、DNN分类器等，都是被预先封装的模型；与sciki-learn中的逻辑回归类似，可以通过一行代码调用。

## 8.8 泰坦尼克号幸存者预测器
该教程中，我们将学习使用TFLearn和TensorFlow，通过泰坦尼克号中乘客的个人信息（如性别、年龄等），对乘客的幸存机会建模。为解决这一传统机器学习问题，我们将要构建一个DNN分类器。

In [11]:
import numpy as np
import tflearn

# 下载titanic数据集：
from tflearn.datasets import titanic
titanic.download_dataset('titanic_dataset.csv')

# 加载CSV文件，并标明第一列代表标签labels：
from tflearn.data_utils import load_csv
data, labels = load_csv('titanic_dataset.csv', target_column=0, categorical_labels=True, n_classes=2)

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Succesfully downloaded titanic_dataset.csv 82865 bytes.


数据需要经过一些预处理，供我们的DNN分类器使用。实际上，必须删除一些分析过程中不需要的列和域。此处删除name和ticket域，因为我们认为，乘客的姓名和票号是不影响其幸存概率的：

In [12]:
def preprocess(data, columns_to_ignore):
    # 预处理过程中，先将数据按照id降序排列，并删除该列：
    for id in sorted(columns_to_ignore, reverse=True):
        [r.pop(id) for r in data]
    for i in range(len(data)):
        # sex域被转换为float型（为方便操作）：
        data[i][1] = 1. if data[i][1] == 'female' else 0.
    return np.array(data, dtype=np.float32)

In [13]:
# 和之前提到过的一样，name和ticket域在分析中会被忽略：
to_ignore=[1, 6]

In [14]:
# 此处调用preprocess过程：
data = preprocess(data, to_ignore)

In [15]:
# 首先，需要指定输入数据的形状。输入样本共有6个特征，需要分批处理样本以节省内存，
# 所以输入数据形状为[None, 6]。None参数表示维数未知，因此可以改变每个批次中处理的样本总个数：
net = tflearn.input_data(shape=[None, 6])

In [16]:
# 最后，使用下列简单的语句构建一个三层神经网络：
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [17]:
# TFLearn提供了一个模型封装器DNN，可以自动完成神经网络分类任务：
model = tflearn.DNN(net)

In [18]:
# 运行10个训练时期，每个批的大小为16：
model.fit(data, labels, n_epoch=10, batch_size=16, show_metric=True)

Training Step: 819  | total loss: 0.46703 | time: 0.395s
| Adam | epoch: 010 | loss: 0.46703 - acc: 0.8026 -- iter: 1296/1309
Training Step: 820  | total loss: 0.46181 | time: 0.402s
| Adam | epoch: 010 | loss: 0.46181 - acc: 0.8099 -- iter: 1309/1309
--


In [19]:
# 最后，评价模型，获取最终准确率：
accuracy = model.evaluate(data, labels, batch_size=16)
print('Accuracy: ', accuracy)

Accuracy:  [0.7868601987159735]
